<a href="https://colab.research.google.com/github/mmv456/Misc/blob/main/Dyno_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Authenticate to GCP, follow given directions

In [115]:
from google.colab import auth
auth.authenticate_user()

Enter project ID

In [116]:
project_id = 'dyno-sequence-api'

Import BigQuery

In [117]:
from google.cloud import bigquery

Set the BigQuery project to the project specified

In [118]:
client = bigquery.Client(project=project_id)

In [119]:
dataset_ref = client.dataset("sequence_data", project=project_id)
dataset = client.get_dataset(dataset_ref)

In [120]:
tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

Sequences_Data


In [121]:
table_ref = dataset_ref.table('Sequences_Data')
table = client.get_table(table_ref)

In [122]:
table.schema

[SchemaField('SeqRecord', 'STRING', 'NULLABLE', None, ()),
 SchemaField('HashOfFile', 'STRING', 'NULLABLE', None, ())]

In [124]:
client.list_rows(table, max_results = 10).to_dataframe()

,SeqRecord,HashOfFile
0,My first sequence data line here,SomeHash
1,My second sequence data line here,SomeHash
2,My second sequence data line here,SomeHash
3,My second sequence data line here,SomeHash
4,My second sequence data line here,SomeHash
5,My second sequence data line here,SomeHash
6,My tgrgertgtrh sequence data line here,SomeHash


Do a sample query into what's in the data

In [126]:
select_all_query = """
                    SELECT *
                    FROM `dyno-sequence-api.sequence_data.Sequences_Data`
                   """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(select_all_query, job_config=safe_config)

print(query_job.to_dataframe())

                                SeqRecord HashOfFile
0        My first sequence data line here   SomeHash
1  My tgrgertgtrh sequence data line here   SomeHash
2       My second sequence data line here   SomeHash
3       My second sequence data line here   SomeHash
4       My second sequence data line here   SomeHash
5       My second sequence data line here   SomeHash
6       My second sequence data line here   SomeHash


Add some data in the table

In [113]:
insert_line_one = """
                    INSERT INTO `dyno-sequence-api.sequence_data.Sequences_Data`
                    VALUES("My tgrgertgtrh sequence data line here", "SomeHash")
                  """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(insert_line_one, job_config=safe_config)

Try to add rows from the file

In [127]:
#from google.colab import auth
#auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'dyno-sequence-api'
!gcloud config set project {project_id}

Updated property [core/project].


In [128]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://seq_data_feb/Parvoviridae_full_130_refseq_sequence.gz /tmp/Parvoviridae_full_130_refseq_sequence.gz

  
# Print the result to make sure the transfer worked.
#!cat /tmp/Parvoviridae_full_130_refseq_sequence.gz

Copying gs://seq_data_feb/Parvoviridae_full_130_refseq_sequence.gz...
/ [1 files][205.5 KiB/205.5 KiB]                                                
Operation completed over 1 objects/205.5 KiB.                                    


In [129]:
from Bio import SeqIO
# Used to convert the fastq stream into a file handle
from io import StringIO
from gzip import open as gzopen
import hashlib

In [130]:
filename="/tmp/Parvoviridae_full_130_refseq_sequence.gz"
fileMD5Hash =""

md5_hash = hashlib.md5()

a_file = open(filename, "rb")
content = a_file.read()
md5_hash.update(content)

digest = md5_hash.hexdigest()
fileMD5Hash=digest

In [131]:
records = SeqIO.parse(
    # There is actually simpler (thanks @peterjc)
    # StringIO(gzopen("random_10.fastq.gz").read().decode("utf-8")),
    gzopen("/tmp/Parvoviridae_full_130_refseq_sequence.gz", "rt"),
    format="fasta")

try:
  for read in records:

    # Only export reads that have a G in positions 7, 8,
    # and 9
    #if read.seq[6:9] == "GGG":
    #print(read.format("fasta"))
    # insert read and fileMD5Hash in two columns
    #try:

      insert_str = """ INSERT INTO `dyno-sequence-api.sequence_data.Sequences_Data`
      VALUES({0}, {1})
      """
      print(insert_str.format("read", "fileMD5Hash"))
      
      print("Success")
      safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
      query_job = client.query(insert_str.format("readfile", "number"), job_config=safe_config)
      print(query_job)
      break
    #except Exception as e:
    #  print(e)
    #  print("fail")

    #exit()
except Exception as e:
  print(e)



 INSERT INTO `dyno-sequence-api.sequence_data.Sequences_Data`
      VALUES(read, fileMD5Hash)
      
Success
